In [5]:
import pandas as pd
import numpy as np
from scipy.optimize import brute

In [48]:
class SMAVectorBacktest:
    def __init__(self, symbol, SMA1, SMA2, start, end):
        self.symbol = symbol
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.start = start
        self.end = end
        self.results = None
        self.get_data()
    
    def get_data(self):
        raw_data = pd.read_csv(
            "https://hilpisch.com/pyalgo_eikon_eod_data.csv",
            index_col=0, parse_dates=True
        )
        # Select your symbol first, THEN drop NaNs (don’t let other tickers shrink your rows)
        raw_data = pd.DataFrame(raw_data[self.symbol]).dropna()
        raw_data = raw_data.loc[self.start:self.end]
        raw_data.rename(columns = {self.symbol:'price'}, inplace=True)
        raw_data['SMA1'] = raw_data['price'].rolling(self.SMA1).mean()
        raw_data['SMA2'] = raw_data['price'].rolling(self.SMA2).mean()
        raw_data['returns'] = np.log(raw_data['price'] / raw_data['price'].shift(1))
        self.data = raw_data

    def set_parameters(self, SMA1=None, SMA2=None):
        if SMA1 is not None:
            self.SMA1 = int(SMA1)
        if SMA2 is not None:
            self.SMA2 = int(SMA2)
        self.data['SMA1'] = self.data['price'].rolling(self.SMA1).mean()
        self.data['SMA2'] = self.data['price'].rolling(self.SMA2).mean()

    def run_strategy(self):
        data = self.data.copy().dropna()
        if data.empty:
            raise ValueError(
                f"No data left after applying SMAs (SMA1={self.SMA1}, SMA2={self.SMA2}). "
                "Use smaller windows or a longer date range."
            )
        data['position'] = np.where(data['SMA1'] > data['SMA2'], 1, -1 )
        data['strategy'] = data['position'].shift(1) * data['returns']
        data.dropna(inplace=True)
        data['creturns'] = data['returns'].cumsum().apply(np.exp)
        data['cstrategy'] = data['strategy'].cumsum().apply(np.exp)
        self.results = data
        # Gross Performance of the strategy
        aperf = data['cstrategy'].iloc[-1]
        # out/underperformance of strategy
        operf = aperf - data['creturns'].iloc[-1]
        return round(aperf, 2), round(operf, 2)
        
    def plot_results(self):
        if self.results is None:
            print("No Results found to plot, Run a strategy first!")
        title = f'{self.symbol} | SMA1={self.SMA1}, SMA2={self.SMA2}'
        self.results[['creturns', 'cstrategy']].plot(title=title, figsize=(10,6))
    
    def update_and_run(self, SMA):
        self.set_parameters(int(SMA[0]), int(SMA[1]))
        return -self.run_strategy()[0]
    
    def optimize_parameters(self, SMA1_range, SMA2_range):
        opt = brute(self.update_and_run, (SMA1_range, SMA2_range), finish=None)
        return opt, -self.update_and_run(opt)

In [49]:
if __name__ == '__main__':
    smabt = SMAVectorBacktest('EUR=', 42, 252,
    '2010-1-1', '2020-12-31')
    print(smabt.run_strategy())
    smabt.set_parameters(SMA1=20, SMA2=100)
    print(smabt.run_strategy())
    print(smabt.optimize_parameters((30, 56, 4), (200, 300, 4)))

(np.float64(1.43), np.float64(0.57))
(np.float64(1.08), np.float64(0.18))
(array([ 50., 240.]), np.float64(1.53))
